In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/penn.csv')

In [4]:
dataset.shape

(8757, 1045)

In [5]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,5,0,3,0,0,0,26,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-01,4,3,0,7,0,0,0,8,0,...,0.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0
2,2018-01-01,5,6,0,1,1,0,2,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0


In [6]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

774

In [7]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 271)

In [8]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

257

In [9]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [10]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 271)
(2190, 271)


In [11]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 257])
torch.Size([2190, 13])
torch.Size([2190, 257])


In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [13]:
bptt = 6

In [14]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [15]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape

(torch.Size([6, 13]), torch.Size([6, 257]), torch.Size([6, 257]))

In [16]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [17]:
class LSTM(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10):
        super().__init__()
        
        # aggregation
        self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
        self.attachment_matrix.requires_grad = True
        
        
        self.hidden_layer_size = hidden_layer_size
        
        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                    torch.zeros(layers,1,self.hidden_layer_size))
        
        lstm_input = communities + feat_size
        self.lstm = nn.LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers)

        #disaggregation
#         self.linear_1 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.linear_2 = nn.Linear(hidden_layer_size, network_size)


    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)

        
        lstm_out, self.hidden_cell = self.lstm(x.view(len(input_seq) ,1, -1), self.hidden_cell)
        
        predictions = self.linear_2(lstm_out.view(len(input_seq), -1))
#         predictions = F.relu(predictions)
#         predictions = self.linear_2(predictions)
        
        return predictions

In [18]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
            model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                            torch.zeros(layers, 1, model.hidden_layer_size))
            prediction.append(model(seq,feat)[-1])

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (r2, rmse, mae)

In [19]:
layers = 1
communities = 24
network_size = len(targetColumns)
feat_size = len(features_cols)

model = LSTM(feat_size = feat_size, hidden_layer_size=communities,
             network_size=network_size, layers=layers,
            communities=communities).to(device)

loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [20]:
epochs = 100


for i in range(epochs):
    model.train()
    for feat,seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq, feat)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        
    scheduler.step()
    if i%10 == 1:
        r2, rmse, mae = evaluate(model)
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 0.85957229 r2: 0.351 rmse: 10.559 mae: 1.036
epoch:  11 loss: 0.75452864 r2: 0.607 rmse: 6.389 mae: 0.860
epoch:  21 loss: 0.75605887 r2: 0.625 rmse: 6.103 mae: 0.863
epoch:  31 loss: 0.75064468 r2: 0.643 rmse: 5.807 mae: 0.859
epoch:  41 loss: 0.74021947 r2: 0.647 rmse: 5.735 mae: 0.848
epoch:  51 loss: 0.74765104 r2: 0.643 rmse: 5.815 mae: 0.842
epoch:  61 loss: 0.75176620 r2: 0.648 rmse: 5.732 mae: 0.838
epoch:  71 loss: 0.74639636 r2: 0.648 rmse: 5.722 mae: 0.841
epoch:  81 loss: 0.76028866 r2: 0.652 rmse: 5.663 mae: 0.840
epoch:  91 loss: 0.75421071 r2: 0.651 rmse: 5.681 mae: 0.825
epoch:  99 loss: 0.7773852348


In [21]:
evaluate(model)

(0.6602214953061789, 5.527703, 0.8185848)

In [22]:
attachment = torch.argmax(F.softmax(model.attachment_matrix, dim=1), dim=1).detach().cpu().numpy()
community_assignment = dict(zip(targetColumns, attachment))
community_assignment

{'1': 6,
 '10': 8,
 '100': 5,
 '101': 0,
 '102': 1,
 '106': 21,
 '107': 9,
 '108': 3,
 '109': 15,
 '11': 23,
 '111': 7,
 '112': 6,
 '113': 5,
 '114': 3,
 '115': 17,
 '116': 0,
 '117': 2,
 '118': 4,
 '119': 0,
 '12': 6,
 '120': 22,
 '121': 0,
 '122': 11,
 '123': 17,
 '124': 1,
 '125': 22,
 '126': 5,
 '127': 10,
 '128': 0,
 '129': 7,
 '13': 1,
 '130': 21,
 '131': 20,
 '132': 6,
 '133': 11,
 '134': 14,
 '135': 23,
 '136': 12,
 '137': 0,
 '138': 6,
 '139': 7,
 '14': 17,
 '140': 12,
 '141': 12,
 '142': 16,
 '143': 3,
 '144': 5,
 '145': 0,
 '146': 2,
 '147': 21,
 '148': 15,
 '149': 17,
 '15': 7,
 '150': 17,
 '151': 23,
 '152': 18,
 '153': 8,
 '154': 17,
 '155': 14,
 '156': 7,
 '157': 4,
 '158': 5,
 '159': 0,
 '16': 22,
 '160': 20,
 '161': 10,
 '162': 3,
 '163': 18,
 '164': 21,
 '165': 17,
 '166': 7,
 '167': 23,
 '168': 1,
 '169': 12,
 '17': 18,
 '170': 19,
 '171': 13,
 '172': 0,
 '173': 5,
 '174': 22,
 '175': 18,
 '176': 4,
 '177': 12,
 '178': 18,
 '179': 17,
 '18': 10,
 '180': 19,
 '181': 2

In [23]:
# 20 comm
# 0.505

In [24]:
# 50 comm
# 